Import all modules you'll need. if you get an error like : no module "module_name" found; just run "pip install module_name" in empty cell.

In [1]:
%matplotlib Qt
%load_ext autoreload 
# to automaticaly reload all the modules that have been modified
%autoreload 2 
import os
from glob import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn import svm, kernel_ridge, metrics, preprocessing
from PIL import Image, ExifTags, ImageFile
import visualize as vis
import preprocessing as pp
import processing as proc
import calculate as cc
import pandas as pd
import correction as corr
from read_WDF import read_WDF, get_exif
import warnings
warnings.filterwarnings('ignore')

For the moment, I haven't found a module in python that reads FTIR .map files, so you need to transform your file into .xlsx

### Reading

In [2]:
os.chdir('C:/Users/raoul.missodey/Desktop/Code_python')
files = glob('*.xlsx')
for i, j in enumerate(files):
    print(i, ') ', j)

0 )  a_3700.xlsx
1 )  FTIR-Carto-T140.xlsx
2 )  FTIR_Carto.xlsx


In [3]:
filename = files[1] # choose the number of .xlsx file you want to use.
# load the file
da =  pd.read_excel(filename)

In [50]:
# show your data
da

,Unnamed: 0,Unnamed: 1,2599.61,2603.47,2607.33,2611.19,2615.04,2618.9,2622.76,2626.61,...,4963.95,4967.81,4971.67,4975.52,4979.38,4983.24,4987.09,4990.95,4994.81,4998.66
0,-2204,1247,0.188377,0.172348,0.160974,0.176721,0.191674,0.186964,0.179221,0.178539,...,-0.114037,-0.117788,-0.119053,-0.116083,-0.118494,-0.121863,-0.121974,-0.119463,-0.118023,-0.119668
1,-2201,1247,0.147287,0.142443,0.120782,0.132604,0.159630,0.156830,0.148951,0.151998,...,-0.038045,-0.042250,-0.040727,-0.038048,-0.042043,-0.044781,-0.043282,-0.042670,-0.044703,-0.045677
2,-2198,1247,0.154792,0.140674,0.126534,0.130258,0.152460,0.156669,0.148612,0.149710,...,0.055802,0.054514,0.055744,0.058381,0.053826,0.050398,0.053213,0.052844,0.049903,0.049343
3,-2195,1247,0.126609,0.128221,0.125939,0.127639,0.137236,0.149622,0.147488,0.143212,...,0.091844,0.091188,0.092735,0.094038,0.092980,0.089317,0.090110,0.090065,0.086781,0.086811
4,-2192,1247,0.142480,0.121166,0.115505,0.136026,0.145826,0.136403,0.126975,0.136495,...,0.111472,0.110103,0.111642,0.114490,0.114219,0.110873,0.111730,0.111440,0.107676,0.107059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,-2138,1280,0.060743,0.041290,0.047987,0.061014,0.057477,0.058508,0.059368,0.061846,...,0.028541,0.025318,0.025248,0.028720,0.026324,0.023912,0.027834,0.029588,0.029432,0.030044
320,-2135,1280,0.045435,0.027787,0.027406,0.049754,0.055192,0.052532,0.055794,0.057292,...,-0.006492,-0.011345,-0.010181,-0.004935,-0.006056,-0.009378,-0.007828,-0.004743,-0.003871,-0.003208
321,-2132,1280,0.048183,0.047353,0.034151,0.037147,0.043654,0.049434,0.059400,0.063431,...,-0.023712,-0.026859,-0.028826,-0.025735,-0.025994,-0.029692,-0.029135,-0.025405,-0.023668,-0.023512
322,-2129,1280,0.050034,0.047908,0.041544,0.053682,0.064766,0.061012,0.060020,0.058751,...,-0.043850,-0.046912,-0.045925,-0.042759,-0.045041,-0.048082,-0.044475,-0.040046,-0.040423,-0.041889


## Preprocessing

if the first row of your data contains the Raman shifts and the first two columns are map_x and map_y. 

In [6]:
da_data = da.iloc[:,2:].values.copy()
ftir_mapx = da.iloc[:,0].values.copy()
ftir_mapy = da.iloc[:,1].values.copy()
raman_shifts = da.columns.values[2:]
raman_shifts =  np.round([float(i) for i in da.columns.values[2:]],1)

if the first row of your data contains the Raman shifts and the last two columns are map_x and map_y. 

In [9]:
da_data = da.iloc[:,0:-2].values.copy()
ftir_mapx = da.iloc[:,-2].values.copy()
ftir_mapy = da.iloc[:,-1].values.copy()
raman_shifts = da.columns.values[:-2]
raman_shifts =  np.round([float(i) for i in da.columns.values[:-2]],1)

In [7]:
# run this cell to see all your spectra
vis.ShowSpectra(da_data, sigma = raman_shifts)

### Baseline Substraction

Two options to substract the baseline 

#### 1st option

In [12]:
#  substract baseline : you have to modify lambda_value(smoothing parameter), p_value(the penalizing weighting factor)
# and lambda1_value(the smoothing parameter for the first derivative of the residual).
check_bline = vis.FindBaseline(da_data,sigma=raman_shifts)

In [12]:
# run to see chosen values
print(f"p: {check_bline.p_val:.2e}, lam: {check_bline.lam_val:.2e},lam1: {check_bline.lam1_val:.2e} ")

p: 1.92e-02, lam: 8.70e+06,lam1: 8.45e-03 


In [13]:
# calculate the basline for all spectrum
baseline = cc.baseline_ials(da_data, p=check_bline.p_val, lam=check_bline.lam_val, lam1= check_bline.lam1_val)

In [16]:
# substraction
db = da_data.copy()
db = db - baseline

#### 2nd option

In [13]:
check_bline = vis.FindBaseline1(da_data,sigma=raman_shifts)

In [14]:
print(f"lam: {check_bline.lam_val:.2e}")

lam: 1.01e+07


In [15]:
# calculate the basline for all spectrum
baseline = cc.baseline_arpls(da_data, lam=check_bline.lam_val)

In [16]:
# substraction
db = da_data.copy()
db = db - baseline

### Normalization

In [17]:
# choose normalize method you want to apply : min_max, l1, l2, max, area, robust_scale 
db -= np.min(db, axis=-1, keepdims=True)
dn = pp.normalize(db, method="min_max")

## Processing

### PCA
Before doing the PCA, you need to know your scan_shape. After that, you can replace it in the "pp.svd_decomp" . You can also changed the "n_components"

In [59]:
# compute pca using svd decomposition
dsvd ,var,score ,l= pp.svd_decomp(da_data,components_sigma=raman_shifts, n_components=10, visualize_clean=1, visualize_components=1,visualize_err=1,scanshape=(12,27))

In [76]:
# advanced svd : click inside the score image to have the related spectra, 
# change the feature_range to normalize the related spectra
advanced_svd = vis.SVD(dn, n_components=10, visualize_clean=1, visualize_components=1,components_sigma=raman_shifts,
                       visualize_err=1,visualize_var=1,feature_range=(-0.075,0.075),scanshape=(12,27))
dsvd ,var,scores,loadings = advanced_svd.svd_element(dn)

### NMF

The same thing for NMF decompostion

In [74]:
# Run this cell to see pure spectrum and pure contribution
dnm = proc.deconvolve_nmf(dn,components_sigma=raman_shifts, n_components=4, visualize_components=1, visualize_compare=1, visualize_err=1,scanshape=(12,27))

Error increased above fractionalctol_increase (C iter). Exiting


In [44]:
# run this cell to fit all your spectra. here we use pseudo-voigt as model.
# left click to draw the model, left click on the top of a model to remove it,
# right click to draw the result of cumulative sum of all your models
# after right clicking, if the sum is not good, just remove your indesirable models et restart.
# scroll your mouse to adjust the width of the models

# Important : double rigth click to end.
#important : your data should have correct baseline et must be normalized.
fiallspectra = vis.FitAllSpectra(dn)

In [45]:
# after adding your model to the peaks position, run this cell to see the results.
# this may take few minutes because I stock the peaks parameters in array which you can access through "fitparams.pic_h"
# the table in the bottom of the plot doesn't change because it's just the parameters of the first fitting.
# by now, it will take time to change the table when updating the plot.
# "fitparams.pic_h" can be used to plot the peak ratio
# "fitparams.pic_a" can be used to plot the peak ratio
from time import*
start = time()
fitparams = vis.FitParams1(dn, fiallspectra.x_size, fiallspectra.peaks_counter,
          fiallspectra.pic, fiallspectra.sum_peak)
end = time()
print((end - start)/60,"min")

0.5224082350730896 min


In [47]:
# run this cell to plot the peak ratio.
# changes values in "ratio" according to which peak over which you whant to plot
# use "fitparams.pic_h" to plot the peak height ratio : for instance ,to plot 1/2 , you will add ratio=(1,2)

# use "fitparams.pic_a" to plot the peak area ratio : for instance ,to plot 1/2 , you will add ratio=(1,2)
# to plot 0/(1+2) , you will add ratio = (0,1,2)

vis.pic_ratio(da,fitparams.pic_h,ratio=(0,1),scanshape=(12,17))